# Gym playground

### Install & import requirements

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
import numpy as np
import gym
from gym.wrappers import Monitor

import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

### Utility functions

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [4]:
import sys
import time
from subprocess import check_output
import os


def combine_video(mp4list):
    path = 'video/final.mp4'
    # if exist, delete
    if os.path.exists(path):
        check_output(["rm", path])
    # First, convert into mpg
    for i, mp4name in enumerate(mp4list):
        mpgname = "./video/part" + str(i) + ".mpg"
        check_output(["ffmpeg", "-i", mp4name, mpgname])
    # Combine
    check_output("cat ./video/part*.mpg > ./video/final.mpg", shell=True)
    # Removing intermediate files
    check_output("rm ./video/part*.mpg", shell=True)
    # Convert into mp4
    check_output(["ffmpeg", "-i", "./video/final.mpg",
                  "-vcodec", "libx264", "-acodec", "libvorbis",
                  path])
    # Removing the last mpg file."
    check_output("rm ./video/final.mpg", shell=True)
    return path

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        # Combining multiple mp4's into one.
        if len(mp4list) > 1:
            mp4 = combine_video(mp4list)
        else:
            mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

show_video()

Could not find video


### Make environment

Monitor automatically save episodes in perfect cubes(i.e. 1, 8, 27, 64,... ).

Refer https://github.com/openai/gym/issues/494

In [5]:
env = Monitor(gym.make("SpaceInvaders-v0"), './video', force=True, video_callable=lambda episode_id: episode_id%10==0)
#env = Monitor(gym.make("SpaceInvaders-v0"), './video', force=True)
#env.render()

def policy(state):
    return np.random.choice(range(6))

for i in range(60):
    print("\r[episode:%6d ]" % i, end="")
    observation = env.reset()
    tot_score = 0
    while True:
        #your agent goes here
        action = policy(observation)
        observation, reward, done, info = env.step(action) 
        tot_score += reward
        if done: 
            break;
env.close()
print()
show_video()

Output hidden; open in https://colab.research.google.com to view.